In [2]:
import matplotlib.pyplot as plt  
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 

import matplotlib.pylab as pylab
# prikaz vecih slika 
pylab.rcParams['figure.figsize'] = 21,15

import numpy as np
import cv2 # OpenCV biblioteka

def show_in_window_and_below(img, below=True):
    if (below):
        plt.imshow(img, 'gray')
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# current image extension
ext = '.JPG'

In [3]:
# runs calculations
def calculate_runs(img):
    runs = [[1] for x in xrange(img.shape[1])] # each column starts with 1 black pixel
    black_runs_flat, white_runs_flat = [], []
    for col in xrange(img.shape[1]): # iterate through all columns
        img[0,col] = 0 # PAINT THE FIRST PIXEL IN PREDEFINED COLOR, to make all columns start the same
        run_index = 0 # start the run
        for row in xrange(1, img.shape[0]): # for each pixel/row in current column
            if (img[row-1 ,col] != img[row, col]):  # if they are not the same,
                # memorize the old run in corresponding array
                if (run_index % 2 == 0): # black run
                    black_runs_flat.append(runs[col][run_index])
                else:
                    white_runs_flat.append(runs[col][run_index])
                # start a new run
                run_index += 1
                runs[col].append(0)
            runs[col][run_index] += 1     # add a pixel to the current run
        # the column ended ==> save the last run for the ended column
        if (run_index % 2 == 0): # black run
            black_runs_flat.append(runs[col][run_index])
        else:
            white_runs_flat.append(runs[col][run_index])
    return runs, black_runs_flat, white_runs_flat

In [4]:
# line thickness and spacings (black and white runs analysis)
from collections import Counter

def calculate_line_thickness(black_flat):
    num_top = 4 # number of top/most common runs
    black_count = Counter(black_flat) # Counter({1: 3, 8: 1, 3: 1, ...})
    m_c_black = black_count.most_common(num_top)
    m_c_black1, m_c_black2 = m_c_black[0][0], m_c_black[1][0]
    print 'Top', num_top, 'most common black runs:', m_c_black
    if (m_c_black1*3 < m_c_black2): # kind of a sanity check
        line_thickness = m_c_black1
    else:
        line_thickness = (m_c_black1 + m_c_black2) / 2.
    print '>>> line thickness:  ', line_thickness
    return line_thickness

def calculate_line_spacing(white_flat, image_height):
    num_top = 4 # number of top/most common runs
    white_count = Counter(white_flat) # print white_count.most_common(50)
    m_c_white = white_count.most_common(num_top)
    m_c_white1, m_c_white2 = m_c_white[0][0], m_c_white[1][0]
    print 'Top', num_top, 'most common white runs', m_c_white
    
    if (m_c_white1 > image_height*0.5): # sanity check
        line_spacing = m_c_white2
    else:
        if (m_c_white2 > image_height*0.5):
            line_spacing = m_c_white1
        else:
            line_spacing = (m_c_white1 + m_c_white2) / 2.
    print 'line spacing: ', line_spacing
    return line_spacing

In [5]:
def remove_staff_lines(img, runs, line_height, staff_thickness_multiplier):
    # copy the image.. python is pass-by-object-reference so it is necessary!
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is deleted/whitened
                if (run_length < line_height * staff_thickness_multiplier):
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %% BIJELI %% RUN = VISINA PRAZNINE +-1
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %%  CRNI  %% RUN = VISINA LINIJE +-1
                    p[cumulative:cumulative + run_length, c] = [255]*(run_length)
            #else: # white runs
            #    do something maybe ?
            cumulative += run_length
    return p

In [21]:
BLACK = 0 # black pixel's color
def remove_staff_lines_with_lines_only(img, runs, line_height, staff_thickness_multiplier, lines_only_img):
    # copy the image.. python is pass-by-object-reference so it is necessary!
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is erased
                if (run_length < line_height * staff_thickness_multiplier):
                    for compare_px in xrange(cumulative, cumulative + run_length):
                        if (lines_only_img[compare_px, c] == BLACK):
                            p[compare_px, c] = 255
                        # p[cumulative:cumulative + run_length, c] = [255]*(run_length)
            cumulative += run_length
    return p

In [7]:
#########
# USE:
#    = in lines-only image, to LOCATE the lines AND/OR check if there is a line on the current location/run
#        - easier to find lines, since there are no other elements
#        - takes more time, since we need to generate the lines-only image,
#           ,BUT THAT IS NOT A PROBLEM SINCE WE WILL NEED IT TO LOCATE THE LINES
#
#    = in binary image, to check if there is a line on the current location/run
#
# da se utvrde linije treba samo odrediti visinu, a ne 
# TIP: mozda za svaki linijski sistem (ili cak liniju) cuvati vise x, koordinata,
#   recimo na svaku petinu sirine slike provjeravati lokacije
#   linijskih sistema (ili pojedinacnih linija) na vise mijesta u slici:
#       |       |        |      |
#       V       V        V      V
# -----..____..--------------------  <== curved line, others are ok
# --------------------------------- 
# ---------------------------------
# ---------------------------------
# ---------------------------------
######
# x je niz od onoliko crnih piksela koliko je prosjecna debljina linije (+-1 ili 2)
# x = [0] * (int(thickness)-1)
# y je niz od onoliko crnih piksela koliko je prosjecna debljina linije + 2 ili 3 ||| INT!
#   ~ pikseli iz y niza pocinju od posljednje tacke posmatrane linije (tj.kandidata za liniju)
# y = current_pos + int(spacing * 0.9 ili 0.8)
# print x_in_y(x, y)
###
def x_in_y(x, y):
    print 'checking', x, 'in', y
    try:
        x_len = len(x)
    except TypeError:
        x_len = 1
        x = type(y)((x,))

    for i in xrange(len(y)):
        if (y[i : i+x_len] == x):
            return True
    return False

# a = [0,0,1,1,1,0,0,0,0]
# b = [1,1,1]
# b = 0 # works also
# print x_in_y(b, a)

In [60]:
# # # # # # # # # # # # # # # # # # # # # # # # # # #
# ALG 2 - look at white runs around the line candidate
# 2*d . . . 1*d . . . X . . . 1*d . . . 2*d
#  0         1        -        2         3
# In order for X to be whitened (erased):
#

def spacing_is_ok(run_length, line_spacing):
#     print '>>> spacing_is_ok <<< run_length: {}, line_spacing: {}'.format(run_length, line_spacing)
    return (run_length < (line_spacing * 1.15)) and (run_length > (line_spacing * 0.85))

def find_lines_up_or_down(column, row_start, black, line_thickness, line_spacing):
#     print 'column', column
#     print 'row_start', row_start, 'black', black, 'line_thickness', line_thickness, 'line_spacing', line_spacing
    scaled_spacing = int(line_spacing*0.9) + 1 # <-- might be the cause of all evil :)
    search_position = row_start + black + scaled_spacing
    # determine the search thickness we are looking for
    search_for_thickness = 1 if (line_thickness <= 3) else 2  # CAUTION <-- might be the cause of all evil :)
    search_for_array = [0] * search_for_thickness # [0]*5 --> [0, 0, 0, 0, 0]
    # determine the search space (exact pixels through which we will search for a line)
    search_height = int(line_thickness)*2 + 1 # thick*2 if (thick <= 3) else thick*1.5 ILI 2.5 ???
    lowest_pixel = search_position + search_height
#     print 'lowest pixel to look at: {} | number of rows: {}'.format(lowest_pixel, len(column))
    if (len(column) > lowest_pixel): # if we have 10 elements, maximum valid index is 9 !
        search_space = column[search_position : lowest_pixel]
        line_below = x_in_y(search_for_array, search_space)
        if (line_below):
            text = 'FOUND a line BELOW\n\tposition: {}|search_thickness: {}|search_height: {}'
#             print text.format(search_position, search_for_thickness, search_height)
            return True
    
    search_position = row_start - scaled_spacing - search_height
#     print 'search position for lines above:', search_position
    lowest_pixel = row_start - scaled_spacing
    if (search_position >= 0):
        search_space = column[search_position : lowest_pixel]
        line_above = x_in_y(search_for_array, search_space)
        if line_above:
            text = 'Found a line ABOVE\n\tposition: {}|search_thickness: {}|search_height: {}'
#             print text.format(search_position, search_for_thickness, search_height)
            return True
    
    return False
# end of find_lines_up_or_down

def run_has_lines_up_or_down(runs_in_column, current_index, line_thickness, line_spacing):
    '''check if there is a line on `line_spacing` above or below the current index '''
    lines = [255,255,255,255]
    return 1

def rm_staff_lines_up_down_neighbours(img, runs, line_thickness, line_spacing, staff_thickness_multiplier):
    '''Removes staff lines by looking at every black run's neighbours, above and below.'''
    # Original image is being changed + Python passes by obj-ref, so it's necessary
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            deleted = False # did we delete the run
            if (r % 2 == 0): # check every black run, shorter than `thickness * multiplier`
                if (run_length < line_thickness * staff_thickness_multiplier):
                    # this is not the last run === check the run AFTER this one
                    if (r + 1 < (len(runs[c]))):
                        if spacing_is_ok(runs[c][r+1], line_spacing): # we found a line - delete it
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
                        # else: # more conditions to add.. like..
                                # check if there is a black run above or below the current one,
                                # on distance that is equal to line_spacing +-1,
                                # whose lenght is = line_thickness +-1
                            
                    # this is not the first run === check the run BEFORE this one
                    if (not deleted and (r - 1 >= 0)):
                        if spacing_is_ok(runs[c][r-1], line_spacing): # we found a line - delete it
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
                    # run_length > 1  ILI_DA_BUDE cumulative > 0  ? ? ? ?? ? ? ? ?? ?
                    if (not deleted): # and run_length > 1)
                        column = img[0:img.shape[0], c].tolist() # convert numpy array to python list
                        if find_lines_up_or_down(column, cumulative, run_length, line_thickness, line_spacing):
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
#                     if run_has_lines_up_or_down(runs[c], r, line_thickness, line_spacing)):
#                 else: # run is thicker than the average line
#                     if (run_length < line_thickness * 2.2): # ASSUMPTION --- for empty note-heads
                        
            cumulative += run_length # !!! # we advanced down for `run_length` pixels !
    return p

In [9]:
# # # # # # # # # # # # # # # # # # # # # # # # # # #
# ALG 3 - compare neighbour points on distance `d`,
# to the left and right, from the observed point X:
# 2*d . . . 1*d . . . X . . . 1*d . . . 2*d
#  0         1        -        2         3
# In order for X to be whitened (erased):
#   - points[1] and points[2] should be black, or
#   - points[0] and points[1] should be black, or
#   - points[2] and points[3] should be black.

def initialize_points(c, distance, img, compare_point, runs):
    '''Gets the neighbours located '''
    points = [255,255,255,255] # initially, pixels are white (maybe -1 if not accessible)
    # 2*left , 1*left , 1*right, 2*right
    if (c - distance >= 0): # we can get the 1*left pixel
        points[1] = img[compare_point, c - distance]
        if (c - 2*distance >= 0): # we can get the 2*left pixel
            points[0] = img[compare_point, c - 2*distance]
    if (c + distance < len(runs)): # we can get the right pixel
        points[2] = img[compare_point, c + distance]
        if (c + 2*distance < len(runs)):
            points[3] = img[compare_point, c + 2*distance]
    return points

def rm_staff_lines_side_neighbours(img, runs, thickness, spacing, thickness_mul, distance):
    if distance > len(runs)/2:
        print 'WHOA! Distance is: {} and there are only {} columns'.format(distance, len(runs))
        return img
    int_thickness = int(thickness)+1
    p = img.copy() # we will erase some lines, so copy the image
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    for c in xrange(len(runs)): # for every column
        cumulative = 0 # number of passed pixels
        for r in xrange(len(runs[c])):
            run_length = runs[c][r]
            if (r % 2 == 0):# for every black run
                if (run_length < thickness * thickness_mul):
                    p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                    cumulative += run_length # !!!
                    continue
                else:
                    pixels_to_remove = int_thickness
                compare_point = cumulative + pixels_to_remove/2 # + run_length/2 # maybe later
                # 4 values of neighbour pixels, some of them must be black,
                #   for deletion of the observed pixel to happen.
                points = initialize_points(c, distance, img, compare_point, runs)
                if (points[1] == 0):
                    if (points[2] == 0):
                        p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
                    else: # out of bounds or white # !! !!  ASSUMPTION !!  !!!
                    # if (points[2] == -1): # more robust? Needs points=[-1,-1,-1,-1]
                        # two to the left is black?
                        if (points[0] == 0):
                            p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
                        # else: NIJE LINIJA :D
                # no black point on the left
                else:
                    if (points[2] == 0 and points[3] == 0):
                        p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
            cumulative += run_length # !!!
    # return the new, processed image
    return p

In [79]:
def parse_image(img, threshold_type, block_size, c_value, staff_thickness_multiplier, params=[]):
    t_t, b_s, s_t_m = threshold_type, block_size, staff_thickness_multiplier
    print('=========\nthreshold_type: {}, block_size: {}, c_value: {}, staff_thickness_multiplier: {}'.format(t_t, b_s, c_value, s_t_m))
    img_ada = cv2.adaptiveThreshold(img, 255, threshold_type, cv2.THRESH_BINARY, block_size, c_value)
    
    dilate_kernel = np.ones((1,30), dtype=np.int) # np.ones((kernel_w, kernel_h), dtype=np.int);
    # staff lines LOCATIONS, along with lines-only image
    lines_only_img, locations = cv2.dilate(img_ada, kernel, iterations=1)
    
    # runs calculation
    runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada)
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # needs image height
    result = rm_staff_lines_up_down_neighbours(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier)
#     distance = int(line_spacing * 0.5)
#     result = rm_staff_lines_side_neighbours(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier, distance)
#     result = rm_s(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier, distance)
    
    cv2.imwrite('./images/dataset/run_X/params_'+str(t_t)+'_'+str(b_s)+'_'+str(c_value)+'_'+str(s_t_m)+ext, result)
    #cv2.imwrite('./images/dataset/run_Y/params_ORIGINAL.jpg', img_ada)

img = cv2.imread('images/dataset/muzikanti'+ext, 0) #  0 -->  read as grayscale
# parse_image(img, cv2.ADAPTIVE_THRESH_MEAN_C, 33, 35, 1.5)

In [249]:
# param values
threshold_types = [cv2.ADAPTIVE_THRESH_MEAN_C]# , cv2.ADAPTIVE_THRESH_GAUSSIAN_C # const values are 0 and 1
block_sizes = [33,35,37,39,43]# [11, 19, 27, 35, 43] # [11,15,19,23,27,31,35, 39, 43]
c_values = [29,33,35,37] # [17,19,21] # [11, 19, 27, 35, 43] # [11,15,19,23,27,31,35, 39, 43]
staff_thickness_multipliers = [1.3, 1.5] # [1., 1.5, 2., 2.5]

In [68]:
for thresh_type in threshold_types:
    for block_size in block_sizes:
        for c_val in c_values:
            for s_t_mul in staff_thickness_multipliers:
                parse_image(img, thresh_type, block_size, c_val, s_t_mul)

threshold_type: 0, block_size: 33, c_value: 29, staff_thickness_multiplier: 1.3
Top two most common black runs: [(4, 36510), (3, 33167)]
>>> line height:   3.5
Top two most common white runs [(21, 24794), (20, 15403)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 29, staff_thickness_multiplier: 1.5
Top two most common black runs: [(4, 36510), (3, 33167)]
>>> line height:   3.5
Top two most common white runs [(21, 24794), (20, 15403)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 33, staff_thickness_multiplier: 1.3
Top two most common black runs: [(3, 38242), (4, 31349)]
>>> line height:   3.5
Top two most common white runs [(21, 26952), (20, 12360)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 33, staff_thickness_multiplier: 1.5
Top two most common black runs: [(3, 38242), (4, 31349)]
>>> line height:   3.5
Top two most common white runs [(21, 26952), (20, 12360)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 35, 

Top two most common black runs: [(4, 36534), (3, 32881)]
>>> line height:   3.5
Top two most common white runs [(21, 25559), (20, 15173)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 33, staff_thickness_multiplier: 1.3
Top two most common black runs: [(3, 37651), (4, 31786)]
>>> line height:   3.5
Top two most common white runs [(21, 27641), (20, 12117)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 33, staff_thickness_multiplier: 1.5
Top two most common black runs: [(3, 37651), (4, 31786)]
>>> line height:   3.5
Top two most common white runs [(21, 27641), (20, 12117)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 35, staff_thickness_multiplier: 1.3
Top two most common black runs: [(3, 39893), (4, 29339)]
>>> line height:   3.5
Top two most common white runs [(21, 28482), (20, 10654)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 35, staff_thickness_multiplier: 1.5
Top two most common black runs: [(3, 39893), (4,

In [185]:
# Test various kernel dimensions on  binarized images with
# morph. operations: erosion, dilation, opening  and  closing
kernel_widths = [1, 2]
kernel_heights = [7, 11, 15]

block_sizes = [51, 51]
c_values = [35, 45]
ada_method = cv2.ADAPTIVE_THRESH_MEAN_C #  GAUS GAUS GAUS GAUS GAUS GAUS !!!
thresh = cv2.THRESH_BINARY
for i in range(1):
    img_path = './images/dataset/viva{}_up.jpg'.format(i+1)
    img = cv2.imread(img_path, 0) # grayscale
    img = cv2.adaptiveThreshold(img, 255, ada_method, thresh, block_sizes[i], c_values[i])
    for kernel_width in kernel_widths:
        for kernel_height in kernel_heights:
            #kernel_width = kernel_height # ONLY FOR TESTING!!!
            kernel = np.ones((kernel_width, kernel_height), dtype=np.int)
            eroded = cv2.erode(img, kernel, iterations=1)
            dilated = cv2.dilate(img, kernel, iterations=1)
            er_b4_dil = cv2.dilate(eroded, kernel, iterations=1)
#             dil_b4_er = cv2.erode(dilated, kernel, iterations=1) # useles...
            
            cv2.imwrite('./images/kernel_2/ER_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                        .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  eroded)
#             cv2.imwrite('./images/kernel_2/DIL_b_{}_c_{}_kw_{}_kh_{}.jpg'\
#                         .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  dilated) # no need to save dil.
            cv2.imwrite('./images/kernel_2/ER_B4_DIL_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                        .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  er_b4_dil)
#             cv2.imwrite('./images/kernel_2/DIL_B4_ER_b_{}_c_{}_kw_{}_kh_{}.jpg'\
#                         .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  dil_b4_er)
# end

In [187]:
# Use all three algorithms on the input image
#   1 - simple:  with no parameters (with optional erosion before the alg.)
#   2 - up-down: basic call, + 3 calls with dilation (kernel_1 1 and 2 iterations, kernel_2)
#   3 - side:    with two distances: line_spacing*A and line_spacing*B. A and B are from [0.1, 2.0]
block_sizes = [51]#, 51] #  ]#
c_values = [45]#, 45]    #  ]#
thresh = cv2.ADAPTIVE_THRESH_MEAN_C
method = cv2.THRESH_BINARY
# img = cv2.imread('images/dataset/muzikanti'+ext, 0) #  0 => read as grayscale
img = cv2.imread('./images/dataset/viva1_up.jpg', 0)

for i in range(len(block_sizes)):
    block = block_sizes[i]
    c = c_values[i]
    img = cv2.adaptiveThreshold(img, 255, thresh, method, block, c)
    path_regular = './images/e_viva/b_{}_c_{}_LINES.jpg'
    cv2.imwrite(path_regular.format(block, c), img)
    # PREPARE image for lines detection  --  ER_B4_DIL_b_51_c_35_kw_2_kh_7
    kernel = np.ones((2, 7), dtype=np.int)
    eroded = cv2.erode(img, kernel)
    er_pa_dil = cv2.dilate(eroded, kernel)
    path_er_pa_dil = './images/e_viva/b_{}_c_{}_ER_B4_DIL_k_2x7.jpg'
    cv2.imwrite(path_er_pa_dil.format(block, c), er_pa_dil)
    img = er_pa_dil
    
    runs, black_runs_flat, white_runs_flat = calculate_runs(img) # runs calculation, for thickness and spacing
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img.shape[0]) # needs image height
    
    thickness_mul = 1.5 # staff_thickness_multiplier
    
    # Remove staff lines - RUNS algorithm ~~Simplest~~ (Aleksandar's)
    rm_s_l = remove_staff_lines(img, runs, line_thickness, thickness_mul)
#     path_regular = './images/e_viva/ER_{}_{}_b_{}_c_{}_RM_S_L.jpg'
#     cv2.imwrite(path_regular.format(k_w, k_h, block, c), rm_s_l)
    path_regular = './images/e_viva/b_{}_c_{}_RM_S_L.jpg'
    cv2.imwrite(path_regular.format(block, c), rm_s_l)
    
    # Remove staff lines - UP-DOWN algorithm (Aleksandar's)
    rm_s_l_up_down = rm_staff_lines_up_down_neighbours(img, runs, line_thickness, line_spacing, thickness_mul)
    path_up_down = './images/e_viva/b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down.format(block, c), rm_s_l_up_down)
    
    up_down_dilated = cv2.dilate(rm_s_l_up_down, kernel)
    path_up_down_dil = './images/e_viva/DIL_{}_{}_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down_dil.format(k_w, k_h, block, c), up_down_dilated)
    
    up_down_dilated_x2 = cv2.dilate(rm_s_l_up_down, np.ones((3, 1), dtype=np.int), iterations=2)
    path_up_down_dil_x2 = './images/e_viva/DIL_3_1_TWO_iters_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down_dil_x2.format(block, c), up_down_dilated_x2)
    
    k_w, k_h = 5, 2 # above kernel is 5 1. This is 5 2
    kernel = np.ones((k_w, k_h), dtype=np.int)
    up_down_dilated = cv2.dilate(rm_s_l_up_down, kernel)
    path_up_down_dil = './images/e_viva/DIL_{}_{}_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down_dil.format(k_w, k_h, block, c), up_down_dilated)
    
    # Remove staff lines - SIDE algorithm (Filip's)
    distance = int(line_spacing * 0.8) ### check for: * 1, * 1.3, * 1.5
    rm_s_l_side = rm_staff_lines_side_neighbours(img, runs, line_thickness, line_spacing, thickness_mul, distance)
    path_side = './images/e_viva/b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
    cv2.imwrite(path_side.format(block, c, distance), rm_s_l_side)
    
    distance = int(line_spacing * 1) ### check for: * 1, * 1.3, * 1.5
    rm_s_l_side = rm_staff_lines_side_neighbours(img, runs, line_thickness, line_spacing, thickness_mul, distance)
    path_side = './images/e_viva/b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
    cv2.imwrite(path_side.format(block, c, distance), rm_s_l_side)
# end


Top 4 most common black runs: [(3, 55472), (4, 16706), (2, 7829), (1, 5105)]
>>> line thickness:   3.5
Top 4 most common white runs [(19, 22786), (18, 13474), (20, 11452), (21, 6751)]
line spacing:  18.5
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 

checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 399 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 348
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 301 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 0]
search position for lines above: 252
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 327 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 276
checking [0, 0] in [0, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 399 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 348
checking [0, 0] in [255, 255

checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 392 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 343
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 441 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 390
checking [0, 0] in [255, 255, 255, 0, 0, 0, 0]
Found a line ABOVE
	position: 390|search_thickness: 2|search_height: 7
lowest pixel to look at: 1121 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 1070
checking [0, 0] in [255, 0, 0, 0, 0, 0, 0]
Found a line ABOVE
	position: 1070|search_thickness: 2|search_height: 7
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 441 | number of ro

lowest pixel to look at: 1158 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 1106
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 1770 | number of rows: 3264
checking [0, 0] in [255, 255, 0, 0, 255, 255, 255]
FOUND a line BELOW
	position: 1763|search_thickness: 2|search_height: 7
lowest pixel to look at: 1791 | number of rows: 3264
checking [0, 0] in [255, 255, 0, 0, 0, 0, 255]
FOUND a line BELOW
	position: 1784|search_thickness: 2|search_height: 7
lowest pixel to look at: 1801 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 1750
checking [0, 0] in [255, 255, 0, 0, 0, 0, 0]
Found a line ABOVE
	position: 1750|search_thickness: 2|search_height: 7
lowest pixel to look at: 1814 | number of rows: 3264
checking [0, 0] in [0, 0, 0, 0, 0, 0, 0]
FOUND a line BELOW
	position: 1807|search_thickness: 2|search_height: 7
lowest pixel to look a

checking [0, 0] in [255, 255, 255, 255, 255, 0, 255]
search position for lines above: 278
checking [0, 0] in [0, 0, 255, 255, 255, 255, 255]
Found a line ABOVE
	position: 278|search_thickness: 2|search_height: 7
lowest pixel to look at: 352 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 303
checking [0, 0] in [0, 0, 255, 255, 255, 255, 255]
Found a line ABOVE
	position: 303|search_thickness: 2|search_height: 7
lowest pixel to look at: 401 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 350
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 1097 | number of rows: 3264
checking [0, 0] in [0, 255, 255, 0, 0, 0, 255]
FOUND a line BELOW
	position: 1090|search_thickness: 2|search_height: 7
lowest pixel to look at: 1120 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 1069
check

...

checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2165
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 2221 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 0, 0, 255]
FOUND a line BELOW
	position: 2214|search_thickness: 2|search_height: 7
lowest pixel to look at: 2228 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 0, 255]
search position for lines above: 2179
checking [0, 0] in [0, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 2244 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2194
checking [0, 0] in [255, 255, 0, 255, 255, 255, 255]
lowest pixel to look at: 2251 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2202
checking [0, 0] in [255, 0, 255, 255, 255, 255, 255]
lowest pixel to look at: 2291 | number of rows: 3264
checking [0, 0] in [255, 255

checking [0, 0] in [0, 255, 255, 255, 255, 255, 0]
search position for lines above: 2366
checking [0, 0] in [0, 0, 0, 0, 0, 0, 0]
Found a line ABOVE
	position: 2366|search_thickness: 2|search_height: 7
lowest pixel to look at: 2424 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 0, 0, 255, 255]
FOUND a line BELOW
	position: 2417|search_thickness: 2|search_height: 7
lowest pixel to look at: 2441 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2391
checking [0, 0] in [0, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 2446 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2396
checking [0, 0] in [255, 255, 255, 0, 255, 255, 0]
lowest pixel to look at: 2484 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2432
checking [0, 0] in [0, 0, 255, 255, 255, 255, 255]
Found a line ABOVE
	position:

lowest pixel to look at: 2913 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 0, 0, 0]
FOUND a line BELOW
	position: 2906|search_thickness: 2|search_height: 7
lowest pixel to look at: 2927 | number of rows: 3264
checking [0, 0] in [0, 255, 255, 255, 0, 0, 0]
FOUND a line BELOW
	position: 2920|search_thickness: 2|search_height: 7
lowest pixel to look at: 2945 | number of rows: 3264
checking [0, 0] in [0, 0, 0, 255, 255, 255, 255]
FOUND a line BELOW
	position: 2938|search_thickness: 2|search_height: 7
lowest pixel to look at: 2965 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2912
checking [0, 0] in [0, 0, 0, 0, 0, 255, 255]
Found a line ABOVE
	position: 2912|search_thickness: 2|search_height: 7
lowest pixel to look at: 3002 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2950
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to 

checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 1999 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 1947
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 2062 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: 2013
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 25 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
search position for lines above: -24
lowest pixel to look at: 987 | number of rows: 3264
checking [0, 0] in [0, 255, 255, 255, 255, 0, 255]
search position for lines above: 935
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 992 | number of rows: 3264
checking [0, 0] in [0, 255, 255, 255, 255, 255, 255]
search position for lines above: 943
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest p

checking [0, 0] in [255, 0, 0, 0, 0, 0, 0]
FOUND a line BELOW
	position: 874|search_thickness: 2|search_height: 7
lowest pixel to look at: 897 | number of rows: 3264
checking [0, 0] in [0, 255, 255, 255, 255, 255, 255]
search position for lines above: 847
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 955 | number of rows: 3264
checking [0, 0] in [0, 0, 255, 255, 255, 255, 0]
FOUND a line BELOW
	position: 948|search_thickness: 2|search_height: 7
lowest pixel to look at: 959 | number of rows: 3264
checking [0, 0] in [255, 255, 0, 255, 255, 255, 255]
search position for lines above: 910
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 964 | number of rows: 3264
checking [0, 0] in [255, 255, 255, 0, 255, 255, 255]
search position for lines above: 914
checking [0, 0] in [255, 255, 255, 255, 255, 255, 255]
lowest pixel to look at: 974 | number of rows: 3264
checking [0, 0] in [255, 255, 0, 0, 0, 0, 255]
FOUND a line BELOW
	posi

In [ ]:
######
# x je niz od onoliko crnih piksela koliko je prosjecna debljina linije (+-1 ili 2)
# x = [0] * (int(thickness)-1)
# y je niz od onoliko crnih piksela koliko je prosjecna debljina linije + 2 ili 3 ||| INT!
#   ~ pikseli iz y niza pocinju od posljednje tacke posmatrane linije (tj.kandidata za liniju)
# y = current_pos + int(spacing * 0.9 ili 0.8)
# print x_in_y(x, y)
###
def x_in_y(x, y):
    print 'checking', x, 'in', y
    try:
        x_len = len(x)
    except TypeError:
        x_len = 1
        x = type(y)((x,))

    for i in xrange(len(y)):
        if (y[i : i+x_len] == x):
            return True
    return False

def find_four_lines_below(column, row_start, black, line_thickness, line_spacing):
    # determine the search position
    # in the given column, from the starting position,
    #   go down for `black` pixels and then go down for line spacing height
    #   then, go back a few pixels, just in case the next line is higher than average
    scaled_spacing = int(line_spacing*0.9) + 1 # <-- might be the cause of all evil :)
    search_position = row_start + black + scaled_spacing
    
    # determine the search thickness we are looking for
    search_for_thickness = 1 if (line_thickness <= 3) else 2  # CAUTION <-- might be the cause of all evil :)
    search_for_array = [0] * search_for_thickness # [0]*5 --> [0, 0, 0, 0, 0]
    
    # determine the search space (exact pixels through which we will search for a line)
    search_height = line_thickness * 2 # thick*2 if (thick <= 3) else thick*1.5 ILI 2.5 ???
    search_space = column[search_position : search_position + search_height]

    # text = 'searching for second line...\n\tposition: {} | search_thickness: {} | search_height: {}'
    # print text.format(search_position, search_for_thickness, search_height)

    # check if there is a line below, by calling the x_in_y function
    second_line = x_in_y(search_for_array, search_space)
    
    if (second_line): print 'WE FOUND A LINE!'
    else: print 'NOT A LINE...'
# end of `find_four_lines_below` method

def locate_lines(lines_only_img, line_thickness, line_spacing):
    '''
    Returns:
        Dictionary of (key, value) pairs where:
          - key is Y coordinate (multiple keys for one detected line),
          - value is a tuple: (staff_number, line_number_in_this_staff)
      '''
    print 'locating_lines....'
    # NO NEED TO CHECK EVERY COLUMN - we can go through only 2,3,4 or 5 (or more)
    # image parts of dimensions: (10px X image_height)  - 10px is experimental value :D
    # for every column      #>>  test - columns 80 to 120
    for col in xrange(80, 120): #>>  lines_only_img.shape[1]
        row = 0 # start from the first pixel in the column
        print '\n| In column', col
        while (row < img.shape[0]):
            black = 0
            # while the pixels
            while (lines_only_img[col][row] == 0):
                black += 1
                # row += 1 # now or as last statement, if we need the starting pixel
            print '\tcounted black pixels: ', black
            if ((black > line_thickness * 0.5) and (black < line_thickness * 1.5)):
                print '\tthis black run is a staff line candidate!'
                # check if there are 4 staff lines below this one
                is_first_staff_line, locations =\
                  find_four_lines_below(lines_only_img[0:lines_only_img.shape[0], col]\
                                        , row, black, line_thickness, line_spacing)
                # If there are 4 staff lines, that means WE FOUND A WHOLE STAFF/STAVE!
                # Go downwards in this column for STAFF_HEIGHT.
                # TWO OPTIONS for getting the staff height:
                #   1) NO NEED TO ADD to the `row` variable! Just set it to the `y` coordinate
                #       of the 5th line of the found STAFF/STAVE/ lines system!
                #   
                #   OR
                #   
                #   2) result of:  last line's y_coordinate MINUS first line y_coorindate
                #      
                #      this is a one-liner, heh..
            else: print '\tthis black run is NOT a line candidate!'
            
            print '\tfinishing column', col, '....'
            # row += ?? #
        
    print 'returnign line locations...'
# end of locate_lines

In [10]:
def dilate_and_save(img, kernel_w, kernel_h):
    '''Dilates a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    kernel = np.ones((kernel_w, kernel_h), dtype=np.int)
    lines_only_img = cv2.dilate(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/dil_{}_{}.jpg'.format(kernel_w, kernel_h)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

def erode_and_save(img, kernel_w_e, kernel_h_e, kernel_w_d=0, kernel_h_d=0):
    '''Erodes a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    if (kernel_w_d == 0):
        dil_str = ''
    else:
        dil_str = '_dil_{}_{}'.format(kernel_w_d, kernel_h_d)
    kernel = np.ones((kernel_w_e, kernel_h_e), dtype=np.int)
    lines_only_img = cv2.erode(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/er_{}_{}{}.jpg'.format(kernel_w_e, kernel_h_e, dil_str)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

In [30]:
def get_lines_only_img(img):
    threshold = cv2.ADAPTIVE_THRESH_MEAN_C
    # # # # # # # # # # # # # # # # # # # # #
    #  ...................................  # ##  TRY WITH BLOCK_C VALUES: ##
    # /|| TRY WITH  35_13  45_11  55_9  ||\ # ##     35_13 45_11 55_9      ##
    # ''''''''''''''''''''''''''''''''''''' #
    method, block, c = cv2.THRESH_BINARY, 55, 9 # 35, 13  || 45, 11
    img_ada = cv2.adaptiveThreshold(img, 255, threshold, method, block, c)

    # dilate with 1x50 --> erode with 2x50  | this has the best results
    kernel_w, kernel_h = 1, 50
    dilated = dilate_and_save(img_ada, kernel_w, kernel_h)
    kernel_w_e, kernel_h_e = 2, 50
#     lines_only_img = erode_and_save(dilated, kernel_w, kernel_h) # same kernel as dilation
    return erode_and_save(dilated, kernel_w_e, kernel_h_e, kernel_w, kernel_h)

In [12]:
def get_line_locations(img):
    '''Locate lines.
    Returns: staff lines LOCATIONS and the lines-only image'''
    lines_only_img = get_lines_only_img(img)
    
    # line locations
    line_locations, x = locate_lines(lines_only_img) # return multiple values??

    
img = cv2.imread('./images/dataset/muzikanti.JPG', 0) # 0 --> read as grayscale
# locate_lines(img)

In [112]:
# # # # # # # ########
# testing for methods
# find_four_lines_below(...) and find_lines_up_or_down(...)

# column = [0]*4 + [255]*20 + [0]*5 + [255]*21 + [0]*4 + [255]*20 # [0,0,0,0,255,19_more_255, 0,0...]
# row_start, black, line_thickness, line_spacing = 23, 5, 4, 20
# row_start, black, line_thickness, line_spacing = 49, 4, 4, 20
# find_lines_up_or_down(column, row_start, black, line_thickness, line_spacing)
# row_start, black, line_thickness, line_spacing = 0, 4, 4, 20
# find_four_lines_below(column, row_start, black, line_thickness, line_spacing)

lowest pixel to look at: 81 | number of rows: 74
search position for lines above: 21
searching for line ABOVE...
	position: 21 | search_thickness: 2 | search_height: 9


True

In [13]:
runs = [[1,2,3], [4,5,6], [7,8,9], [10,11,12], [13,14,15]]
iiii = 2
print runs[iiii-1 : iiii+2] # gets the runs on the left and right
xxx = np.ones((5,5),dtype=np.int)
xxx[0:5, 2].tolist()

[[4, 5, 6], [7, 8, 9], [10, 11, 12]]


[1, 1, 1, 1, 1]

In [ ]:
def last_test():
#     img = cv2.imread('./images/dataset/sviraj_up.jpg', 0) # read the image as GRAYcscale
    root_path = './images/dataset/jamscik'
    img = cv2.imread(root_path + '/jamscik_gray_up.jpg', 0) # read the image as GRAYcscale
    
    adaptiveMethod, thresholdType, blockSize, C = cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 35
    img_ada = cv2.adaptiveThreshold(img, 255, adaptiveMethod, thresholdType, blockSize, C)
#     img_ada = cv2.erode(img_ada, np.ones((1,7), dtype=np.int)) # thickens the lines
    
    runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # calculation needs image height
    
    lines_only_img = get_lines_only_img(img) # ...
    # line_thickness = 3 # CHEAT LINE
    erode_width = int(line_thickness+1) # set to be 4 allways?
    lines_only_img = cv2.erode(lines_only_img, np.ones((erode_width, 1), dtype=np.int)) # thickens the lines
    
    thickness_mul = 1.5 # staff_thickness_multiplier
#     thickness_mul = 2 # staff_thickness_multiplier
    
#     we can send the grayscale image, also :D  Just send `img` instead of `img_ada`
    result = remove_staff_lines_with_lines_only(img_ada, runs, line_thickness, thickness_mul, lines_only_img)
    result_path = root_path + '/RM_S_L_LINES_ONLY_b{}_c{}_ada{}_er_kw{}.jpg'.format(blockSize, C, adaptiveMethod, erode_width)
    cv2.imwrite(result_path, result)
    
    runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # needs image height
    
    thickness_mul = 1.5 # staff_thickness_multiplier
    
    # Remove staff lines - RUNS algorithm ~~Simplest~~ (Aleksandar's)
    rm_s_l = remove_staff_lines(img_ada, runs, line_thickness, thickness_mul)
    path_regular = root_path + '/b_{}_c_{}_RM_S_L.jpg'
    cv2.imwrite(path_regular.format(blockSize, C), rm_s_l)
    
    # Remove staff lines - UP-DOWN algorithm (Aleksandar's)
    rm_s_l_up_down = rm_staff_lines_up_down_neighbours(img_ada, runs, line_thickness, line_spacing, thickness_mul)
    path_up_down = root_path + '/b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down.format(blockSize, C), rm_s_l_up_down)

last_test()